In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [33]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, scale!, evaluate
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using Roots
using NLsolve

### Inverse of Integrated Function

In [23]:
Nx = 2
Ne = 8
m = 5
ens = EnsembleState(Nx, Ne)

ens.S .=  [0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]';

X = ens.S 
idx = [0 0; 0 1; 1 0; 1 1]

B = MultiBasis(BasisProHermite(6; scaled =true), Nx)

coeff = [ -0.00319245982105743; 0.8024007165102715; -0.40077744068752075; -1.0563328221249926]

f = ParametricFunction(ExpandedFunction(B, idx, coeff))
R = IntegratedFunction(f);

# F = evaluate(R, X)

F = [  1.4856969015693695;
       0.5978925425658191;
       0.4593757680535011;
       1.1805056241296106;
       0.5264028580778785;
       1.0159025856390884;
      -0.9829981708131781;
      -1.098528178152973];

# Naive approach
Xd = zeros(Ne)
@time for i=1:Ne
    Xd[i] = find_zero(t->evaluate(R, reshape([X[1,i]; t],(2,1)))[1]-F[i], (-5.0, 5.0), Roots.Brent())
end

norm(Xd - X[2,:])<1e-5

  0.084172 seconds (205.49 k allocations: 9.818 MiB)


true

In [32]:
Xdm1 = deepcopy(X)
Xdm1[2,:] .= zeros(Ne)
ψoff = evaluate_offdiagbasis(R.f, Xdm1)
ψdiag = repeated_evaluate_basis(R.f.f, zeros(Ne))

f0 = (ψoff .* ψdiag)*coeff



8-element Array{Float64,1}:
 -0.04323724553290472
 -0.057675939687330875
  0.13404031081295917
  0.1140224093452807
  0.02766892653662287
 -0.06448416810855422
 -0.13791826613141275
 -0.10692617736105602

In [42]:
@time F(1.0)

  0.000005 seconds (1 allocation: 16 bytes)


-0.150696049819974

In [ ]:
@time F = cumsum(Fun(t->f([1.0;t]), 0..10))

In [27]:
Rf = t->evaluate(R, reshape([1.0; t],(2,1)))

#5 (generic function with 1 method)

In [32]:
@time Rf(2.0)

  0.000101 seconds (701 allocations: 27.234 KiB)


1-element Array{Float64,1}:
 1.8855911450121758

### Greedy fit

In [18]:
Nx = 2
Ne = 8
m = 5
ens = EnsembleState(Nx, Ne)

ens.S .=  [0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]';
X = ens.S

Hk_old = HermiteMapk(m, Nx; α = 1e-6)

@time S = Storage(Hk_old.I.f, X)

  0.000042 seconds (98 allocations: 4.672 KiB)


Storage{5,1,2}(ParametricFunction{5,1,2}(ExpandedFunction{5,1,2}(MultiBasis{5,2}(Basis of 5 functions:
ProHermite{0}(ProPolyHermite{0}(ImmutablePolynomial(0.6316187777460647), true), true)
ProHermite{1}(ProPolyHermite{1}(ImmutablePolynomial(0.6316187777460647*x), true), true)
ProHermite{2}(ProPolyHermite{2}(ImmutablePolynomial(-0.44662192086900115 + 0.44662192086900115*x^2), true), true)
ProHermite{3}(ProPolyHermite{3}(ImmutablePolynomial(-0.7735718587191167*x + 0.25785728623970555*x^3), true), true)
ProHermite{4}(ProPolyHermite{4}(ImmutablePolynomial(0.38678592935955836 - 0.7735718587191167*x^2 + 0.12892864311985278*x^4), true), true)
), [0 0], [0.0])), [0.6204340285425334; 0.6109307382763318; … ; 0.3833382109324337; 0.5453070915611171], [0.37876306502903495; 0.575998908561622; … ; 0.5115835457601993; 0.4908094965408497], [0.6316187777460647; 0.6316187777460647; … ; 0.6316187777460647; 0.6316187777460647], [-0.270855361617588; -0.17488018062621116; … ; 0.23486928481737188; 0.246496799

In [73]:
update_component(Hk_old, X, getreducedmargin([0 0]), S)

([0 0; 1 0], [0 1; 2 0])

In [67]:
J = 0.0
dJ = zeros(1)
J = negative_log_likelihood!(J, dJ, getcoeff(Hk_old), S, Hk_old, X)
dJ

1-element Array{Float64,1}:
 0.12925328841093559

In [68]:
B = MultiBasis(BasisProHermite(m-1; scaled =true), Nx)
f00 = ParametricFunction(ExpandedFunction(B, [0 0], [1.0]))
f10 = ParametricFunction(ExpandedFunction(B, [1 0], [1.0]))

ParametricFunction{5,1,2}(ExpandedFunction{5,1,2}(MultiBasis{5,2}(Basis of 5 functions:
ProHermite{0}(ProPolyHermite{0}(ImmutablePolynomial(0.6316187777460647), true), true)
ProHermite{1}(ProPolyHermite{1}(ImmutablePolynomial(0.6316187777460647*x), true), true)
ProHermite{2}(ProPolyHermite{2}(ImmutablePolynomial(-0.44662192086900115 + 0.44662192086900115*x^2), true), true)
ProHermite{3}(ProPolyHermite{3}(ImmutablePolynomial(-0.7735718587191167*x + 0.25785728623970555*x^3), true), true)
ProHermite{4}(ProPolyHermite{4}(ImmutablePolynomial(0.38678592935955836 - 0.7735718587191167*x^2 + 0.12892864311985278*x^4), true), true)
), [1 0], [1.0]))

In [57]:
evaluate_offdiagbasis(f00, X)

8×1 Array{Float64,2}:
 0.6204340285425334
 0.6109307382763318
 0.43086165657565983
 0.22030089630812552
 0.6263833997724193
 0.6052629336645401
 0.3833382109324337
 0.5453070915611171

In [58]:
evaluate_diagbasis(f00, X) #- ProHermite(0; scaled = true).(X[2,:])

8×1 Array{Float64,2}:
 0.37876306502903495
 0.575998908561622
 0.6218825550713736
 0.5121880772517214
 0.6048909636466151
 0.4877759213219178
 0.5115835457601993
 0.4908094965408497

In [59]:
evaluate(f00, X)

8-element Array{Float64,1}:
 0.2349974942990817
 0.351895438453913
 0.267945347873556
 0.11283549249688966
 0.37889365830058164
 0.2952326851102279
 0.19610952117418562
 0.26764189906926694

In [60]:
evaluate_offdiagbasis(f10, X) #- ProHermite(1; scaled = true).(X[1,:])

8×1 Array{Float64,2}:
  0.16586249015236107
  0.22297688992535733
 -0.532945708868131
 -0.4521896197620584
 -0.11429304703947449
  0.24991730215062427
  0.5417795602750272
  0.41805804582405576

In [61]:
evaluate_diagbasis(f10, X) #- ProHermite(0; scaled = true).(X[2,:])

8×1 Array{Float64,2}:
 0.37876306502903495
 0.575998908561622
 0.6218825550713736
 0.5121880772517214
 0.6048909636466151
 0.4877759213219178
 0.5115835457601993
 0.4908094965408497

In [65]:
getreducedmargin([0 0])

2×2 Array{Int64,2}:
 0  1
 1  0

In [62]:
evaluate(f10, X)

8-element Array{Float64,1}:
  0.0628225851434564
  0.12843444523147074
 -0.3314296391452377
 -0.23160613189911572
 -0.06913483136181563
  0.12190364231080887
  0.2771655084659
  0.20518685899575628

In [9]:
@time Hk_new, train_error, valid_error = greedyfit(Hk_old, X, 6; verbose = true);

train_error[1] = 1.3172777885451104
reduced_margin = [0 1; 1 0]
coeff0 = [0.0]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
count = 2
idx_new = [0 0; 1 0]
reduced_margin = [0 1; 2 0]
S.f = ParametricFunction{5,2,2}(ExpandedFunction{5,2,2}(MultiBasis{5,2}(Basis of 5 functions:
ProHermite{0}(ProPolyHermite{0}(ImmutablePolynomial(0.6316187777460647), true), true)
ProHermite{1}(ProPolyHermite{1}(ImmutablePolynomial(0.6316187777460647*x), true), true)
ProHermite{2}(ProPolyHermite{2}(ImmutablePolynomial(-0.44662192086900115 + 0.44662192086900115*x^2), true), true)
ProHermite{3}(ProPolyHermite{3}(ImmutablePolynomial(-0.7735718587191167*x + 0.25785728623970555*x^3), true), true)
ProHermite{4}(ProPolyHermite{4}(ImmutablePolynomial(0.38678592935955836 - 0.7735718587191167*x^2 + 0.12892864311985278*x^4), true), true)
), [0 0; 1 0], [-1.3911852989111255, 0.0]))
coeff0 = [-1.39118

In [10]:
getidx(Hk_new)

6×2 Array{Int64,2}:
 0  0
 1  0
 2  0
 0  1
 3  0
 1  1

In [10]:
train_error

7-element Array{Float64,1}:
 1.3172777885451104
 1.2274888293485362
 1.1142726775809417
 1.046903969351235
 0.9895934271488671
 0.8502240638418765
 0.6209131553307563

In [187]:
negative_log_likelihood!(0.0, nothing, getcoeff(Hk_old), S, Hk_old, X)
getreducedmargin(zeros(Int64,0,2))

0×0 Array{Int64,2}

In [185]:
S = Storage(Hk_old.I.f, X)
coeff0 = getcoeff(Hk_old)
@show coeff0

res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, Hk_old, X)), coeff0, Optim.BFGS())
Optim.minimizer(res)
res

coeff0 = [0.0]


 * Status: success

 * Candidate solution
    Final objective value:     1.227489e+00

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 3.89e-03 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.80e-03 ≰ 0.0e+00
    |f(x) - f(x')|         = 7.02e-07 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.72e-07 ≰ 0.0e+00
    |g(x)|                 = 6.29e-11 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    2
    f(x) calls:    8
    ∇f(x) calls:   8


In [183]:
evaluate(Hk_old.I, X)

8-element Array{Float64,1}:
  1.43021
  0.607224
  0.249277
  0.915629
  0.415874
  1.01672
 -0.918205
 -1.00445

train_error[1] = 1.3172777885451104
reduced_margin = [0 1; 1 0]
coeff0 = [0.0]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validatio

coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852

Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res)

getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3

0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training e

...

coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852

Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res)

getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3

Excessive output truncated after 524347 bytes.


coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.3911852989111255]
Optim.minimizer(res) = [-1.3911852989111255]
getcoeff(Hk) = [-1.3911852989111255]
0 terms - Training error: 1.2274888293485362, Validation error: 1.0729544313674289
coeff0 = [-1.391185

In [13]:
    Nx = 2
    Ne = 8
    m = 5
    ens = EnsembleState(Nx, Ne)

    ens.S .=  [0.267333   1.43021;
              0.364979   0.607224;
             -1.23693    0.249277;
             -2.0526     0.915629;
             -0.182465   0.415874;
              0.412907   1.01672;
              1.41332   -0.918205;
              0.766647  -1.00445]';
    X = ens.S

    idx = [0 0; 0 1; 1 0; 0 2; 2 0; 1 1]

    coeff = [ -0.9905841755746164;
          0.6771992097558741;
         -2.243695806805015;
         -0.34591297359447354;
         -1.420159186008486;
         -0.5361337327704369]

    Hk_old = HermiteMapk(m, Nx, idx, coeff; α = 1e-6);

    S = Storage(Hk_old.I.f, X)

    dJ_old = zeros(6)
    J_old = 0.0
    @time J_old = negative_log_likelihood!(J_old, dJ_old, getcoeff(Hk_old), S, Hk_old, X)

    @test abs(J_old - 1.639729324892425)<1e-5

    dJt_old  = [0.078019218557750;
               0.034817260498187;
              -0.181070374255429;
              -0.084058665820870;
              -0.073995221010768;
               0.051631902496933]

    @test norm(dJ_old - dJt_old)<1e-5



    reduced_margin0 = getreducedmargin(getidx(Hk_old))
    idx_old0 = getidx(Hk_old)
    idx_new0 = vcat(idx_old0, reduced_margin0)

    # Define updated map
    f_new = ExpandedFunction(Hk_old.I.f.f.B, idx_new0, vcat(getcoeff(Hk_old), zeros(size(reduced_margin0,1))))
    Hk_new = HermiteMapk(f_new; α = 1e-6)
    idx_new, reduced_margin = update_component(Hk_old, X, reduced_margin0, S)

    dJ_new = zeros(10)
    J_new = 0.0
    S = update_storage(S, X, reduced_margin0)
    J_new = negative_log_likelihood!(J_new, dJ_new, getcoeff(Hk_new), S, Hk_new, X)

     Jt_new  = 1.639729324892425
    dJt_new  = [ 0.078019218557750;
                 0.034817260498187;
                -0.181070374255429;
                -0.084058665820870;
                -0.073995221010768;
                 0.051631902496933;
                -0.084059299875320;
                 0.134756912666511;
                 0.000251818300919;
                 0.048468510410649]

    @test abs(J_new -    Jt_new)<1e-5
    @test norm(dJ_new - dJt_new)<1e-5


    @test findmax(abs.(dJ_new[7:10]))[2] == 8-6

    @test idx_new0[8,:] == reduced_margin0[2,:]

    @test reduced_margin == updatereducedmargin(getidx(Hk_old), reduced_margin0, 2)[2]
    @test idx_new == updatereducedmargin(getidx(Hk_old), reduced_margin0, 2)[1]

  0.000129 seconds (929 allocations: 53.328 KiB)


Test Passed

In [78]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, Hk_old, ens.S)), coeff, Optim.BFGS())
coeffopt = Optim.minimizer(res)

1-element Array{Float64,1}:
 -1.144117388080984

In [ ]:
greedyfit